In [186]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from zipfile import ZipFile
from datetime import date, timedelta, datetime

In [187]:
print(str(datetime.now()) + " Running 6_COVID_14day_Metrics_Process")
yearDate = date.today().strftime('%m%d%y')
print("today: "+yearDate)
yearDate_long = date.today().strftime('%m/%d/%y')
print(yearDate_long)
yesterday = date.today() - timedelta(days=1)
yearDate_yesterday = yesterday.strftime('%m%d%y')
print("yesterday: "+yearDate_yesterday)

2020-11-16 12:54:13.908524 Running 6_COVID_14day_Metrics_Process
today: 111620
11/16/20
yesterday: 111520


In [188]:
covid_cases_file = "data/Florida_COVID19_Cases_by_County{}.xlsx".format(yearDate)
df_covid_cases = pd.read_excel(covid_cases_file, sheet_name = 'Florida_COVID19_Cases_by_County')
df_covid_case = df_covid_cases.drop(columns=['Unnamed: 0'],axis=1)
df_covid_case.head()

,OBJECTID,OBJECTID_12_13,DEPCODE,COUNTY,COUNTYNAME,County_1,State,PUIsTotal,Age_0_4,Age_5_14,...,EverMon,MonNow,Shape__Area,Shape__Length,GlobalID,NewPos,NewNeg,NewTested,NewPercPos,MedianAge
0,66,66,1,1,ALACHUA,Alachua,FL,26643,416,691,...,1770,1747,2.510756e+09,272135.471452,9eddb8fb-00dd-45f1-a09d-8e689049e28a,86,1855,1941,0.044307,24
1,17,17,2,3,BAKER,Baker,FL,3163,71,154,...,2226,2225,1.520246e+09,183737.354704,393e3bc9-079e-4ad6-9162-d77ee72feb59,8,120,128,0.062500,41
2,15,15,3,5,BAY,Bay,FL,11176,139,340,...,2044,2037,1.930771e+09,558412.990826,a66f3b06-8ee0-4dec-843e-f41d712aad2f,126,947,1073,0.117428,42
3,40,40,4,7,BRADFORD,Bradford,FL,2977,70,161,...,1899,1868,7.767446e+08,167178.838253,588f1d06-056b-4af5-a2e7-a5366c833ba5,8,97,105,0.076190,46
4,42,42,5,9,BREVARD,Brevard,FL,32581,422,1006,...,3251,3193,2.687644e+09,793218.333609,943db4a8-472c-444e-b81a-0e7cf9758ca2,140,2215,2355,0.059448,44


In [189]:
df_new_cases = df_covid_cases[['County_1','NewPos']].copy()
df_new_cases.head()

,County_1,NewPos
0,Alachua,86
1,Baker,8
2,Bay,126
3,Bradford,8
4,Brevard,140


In [190]:
df_new_tested = df_covid_cases[['County_1','NewTested']].copy()
df_new_tested.head()

,County_1,NewTested
0,Alachua,1941
1,Baker,128
2,Bay,1073
3,Bradford,105
4,Brevard,2355


In [191]:
result_folder = 'results/{}'.format(yearDate)
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

In [192]:
hosp_file = os.path.join(result_folder,"FL_County_Hospital_ICU{}.xlsx".format(yearDate))
df_hosps = pd.read_excel(hosp_file, sheet_name = 'Sheet1')
df_hosp = df_hosps.drop(columns=['Unnamed: 0'],axis=1)
df_hosp.head()

,County,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity,Available,Available Capacity,Bed Census,Total Staffed Bed Capacity,COVID Hospitalizations
0,ALACHUA,251,57,0.185065,8,0.170213,39,308,47,287,0.164282,1460,1747,83
1,BAKER,0,0,0.000000,0,0.000000,0,0,0,114,0.182400,511,625,0
2,BAY,60,0,0.000000,2,0.500000,2,60,4,100,0.181818,450,550,24
3,BRADFORD,0,0,0.000000,0,0.000000,0,0,0,5,0.500000,5,10,0
4,BREVARD,124,43,0.257485,0,0.000000,0,167,0,506,0.312346,1114,1620,79


In [193]:
df_new_hosp = df_hosp[['County', 'COVID Hospitalizations']].copy()
df_new_hosp.head()

,County,COVID Hospitalizations
0,ALACHUA,83
1,BAKER,0
2,BAY,24
3,BRADFORD,0
4,BREVARD,79


In [194]:
covid_update_file = os.path.join('results/{}'.format(yearDate_yesterday),"COVID_daily_update{}.xlsx".format(yearDate_yesterday))
df_NewPositive0 = pd.read_excel(covid_update_file, sheet_name = 'NewPositive')
df_NewTested0 = pd.read_excel(covid_update_file, sheet_name = 'NewTested')
df_Hospitalization0 = pd.read_excel(covid_update_file, sheet_name = 'Hospitalization')

In [195]:
# remove first unnamed column
df_Hospitalization = df_Hospitalization0.drop(columns=['Unnamed: 0'],axis=1)
df_NewPositive = df_NewPositive0.drop(columns=['Unnamed: 0'],axis=1)
df_NewTested = df_NewTested0.drop(columns=['Unnamed: 0'],axis=1)

In [196]:
df_NewPositive.head()

,DEPCODE,COUNTY,COUNTYNAME,County_1,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,...,11/06/2020,11/07/2020,11/08/2020,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20
0,1,1,ALACHUA,Alachua,114,86,146,160,176,172,...,48,24,43,130,75,91,94,75,17,17
1,2,3,BAKER,Baker,10,9,11,20,15,13,...,4,2,5,5,12,4,8,9,5,5
2,3,5,BAY,Bay,13,27,12,21,14,37,...,27,19,30,80,58,67,36,51,61,61
3,4,7,BRADFORD,Bradford,4,6,18,23,15,17,...,4,1,7,4,8,10,3,2,1,1
4,5,9,BREVARD,Brevard,36,57,47,95,72,90,...,109,145,111,96,148,172,70,145,145,145


In [197]:
df_NewPositive_merge = pd.merge(df_NewPositive, df_new_case, left_on=['County_1'],right_on= ['County_1'], how = 'outer')
df_NewPositive_new = df_NewPositive_merge.rename(columns={"NewPos":yearDate_long})
df_NewPositive_new.head()

,DEPCODE,COUNTY,COUNTYNAME,County_1,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,...,11/07/2020,11/08/2020,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20
0,1,1,ALACHUA,Alachua,114,86,146,160,176,172,...,24,43,130,75,91,94,75,17,17,86
1,2,3,BAKER,Baker,10,9,11,20,15,13,...,2,5,5,12,4,8,9,5,5,8
2,3,5,BAY,Bay,13,27,12,21,14,37,...,19,30,80,58,67,36,51,61,61,126
3,4,7,BRADFORD,Bradford,4,6,18,23,15,17,...,1,7,4,8,10,3,2,1,1,8
4,5,9,BREVARD,Brevard,36,57,47,95,72,90,...,145,111,96,148,172,70,145,145,145,140


In [198]:
df_NewTested_merge = pd.merge(df_NewTested, df_new_tested, left_on=['County_1'],right_on= ['County_1'], how = 'outer')
df_NewTested_new = df_NewTested_merge.rename(columns={"NewTested":yearDate_long})
df_NewTested_new.head()

,DEPCODE,COUNTY,COUNTYNAME,County_1,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,...,11/07/2020,11/08/2020,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20
0,1,1,ALACHUA,Alachua,955,1290,1016,1426,2055,2202,...,684,614,3646,1121,1901,1787,1158,432,432,1941
1,2,3,BAKER,Baker,54,60,68,67,235,124,...,36,101,98,178,119,82,60,32,32,128
2,3,5,BAY,Bay,194,93,273,314,167,359,...,319,352,211,771,623,587,463,455,455,1073
3,4,7,BRADFORD,Bradford,63,73,75,96,114,143,...,11,51,49,69,127,40,71,112,112,105
4,5,9,BREVARD,Brevard,1363,1133,671,1440,1806,1579,...,1335,1923,1107,1342,1288,2115,1457,1079,1079,2355


In [199]:
df_Hospitalization_merge = pd.merge(df_Hospitalization, df_new_hosp, left_on=['County'],right_on= ['County'], how='outer')
df_Hospitalization_new = df_Hospitalization_merge.rename(columns={"COVID Hospitalizations":yearDate_long})
df_Hospitalization_new.head()

,ID,County,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,2020-09-20 00:00:00,2020-09-21 00:00:00,...,11/07/2020,11/08/2020,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20
0,0,ALACHUA,87,83,89,77,68,70,69,72,...,64,69,71,73,74,74,69,70,72,83
1,1,BAKER,21,12,12,15,16,16,16,13,...,0,1,1,1,0,1,1,1,0,0
2,2,BAY,24,25,26,29,27,27,28,24,...,18,19,19,23,22,31,28,29,23,24
3,3,BRADFORD,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,BREVARD,58,53,46,42,44,50,46,43,...,72,71,73,81,72,74,78,72,73,79


In [200]:
# Export updated covid data to excel file
print("Saving COVID_daily_update{}.xlsx in results/".format(yearDate))
with pd.ExcelWriter(os.path.join(result_folder,'COVID_daily_update{}.xlsx'.format(yearDate))) as writer:  
    df_NewPositive_new.to_excel(writer, sheet_name='NewPositive')
    df_NewTested_new.to_excel(writer, sheet_name='NewTested')
    df_Hospitalization_new.to_excel(writer, sheet_name='Hospitalization')

Saving COVID_daily_update111620.xlsx in results/


In [201]:
df_NewPositive_new['7dayPos'] = df_NewPositive_new.iloc[:,-7:].sum(axis=1)
df_NewPositive_new['14dayPos'] = df_NewPositive_new.iloc[:,-15:-1].sum(axis=1)
df_NewPositive_new

,DEPCODE,COUNTY,COUNTYNAME,County_1,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,...,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,7dayPos,14dayPos
0,1,1,ALACHUA,Alachua,114,86,146,160,176,172,...,130,75,91,94,75,17,17,86,455,914
1,2,3,BAKER,Baker,10,9,11,20,15,13,...,5,12,4,8,9,5,5,8,51,96
2,3,5,BAY,Bay,13,27,12,21,14,37,...,80,58,67,36,51,61,61,126,460,756
3,4,7,BRADFORD,Bradford,4,6,18,23,15,17,...,4,8,10,3,2,1,1,8,33,60
4,5,9,BREVARD,Brevard,36,57,47,95,72,90,...,96,148,172,70,145,145,145,140,965,1856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,129,WAKULLA,Wakulla,19,14,8,12,8,14,...,1,20,12,11,15,3,3,24,88,134
65,66,131,WALTON,Walton,17,9,14,5,3,8,...,16,22,20,34,39,18,18,37,188,666
66,67,133,WASHINGTON,Washington,0,2,3,0,1,3,...,9,8,1,2,22,13,13,25,84,136
67,998,998,A State,State,1827,2701,2340,3309,3239,3603,...,3876,4316,5719,5534,6787,4452,4452,9928,41188,76457


In [202]:
df_NewTested_new['7dayTested'] = df_NewTested_new.iloc[:,-7:].sum(axis=1)
df_NewTested_new['14dayTested'] = df_NewTested_new.iloc[:,-15:-1].sum(axis=1)
df_NewTested_new

,DEPCODE,COUNTY,COUNTYNAME,County_1,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,...,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,7dayTested,14dayTested
0,1,1,ALACHUA,Alachua,955,1290,1016,1426,2055,2202,...,3646,1121,1901,1787,1158,432,432,1941,8772,19083
1,2,3,BAKER,Baker,54,60,68,67,235,124,...,98,178,119,82,60,32,32,128,631,1253
2,3,5,BAY,Bay,194,93,273,314,167,359,...,211,771,623,587,463,455,455,1073,4427,7254
3,4,7,BRADFORD,Bradford,63,73,75,96,114,143,...,49,69,127,40,71,112,112,105,636,1054
4,5,9,BREVARD,Brevard,1363,1133,671,1440,1806,1579,...,1107,1342,1288,2115,1457,1079,1079,2355,10715,20273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,129,WAKULLA,Wakulla,126,76,58,198,129,208,...,28,154,167,148,158,66,66,317,1076,1892
65,66,131,WALTON,Walton,114,112,148,144,142,203,...,104,195,175,225,163,74,74,394,1300,3984
66,67,133,WASHINGTON,Washington,75,26,25,67,18,52,...,13,42,23,32,52,24,24,91,288,729
67,998,998,A State,State,47014,64194,52537,74522,77646,79780,...,48365,52115,73420,75557,85600,45072,45072,131506,508342,1016668


In [203]:
df_Hospitalization_new.iloc[:,-1]

0       83
1        0
2       24
3        0
4       79
5      284
6        0
7       44
8       47
9       29
10      64
11      16
12       7
13     180
14      71
15      11
16       0
17       0
18       0
19       0
20       0
21      57
22      68
23     224
24       1
25      14
26      22
27      49
28      73
29      38
30       0
31      34
32      54
33      13
34     465
35      10
36       3
37      47
38       4
39     163
40      85
41     235
42      75
43     156
44      99
45      10
46      13
47     102
48      47
49      10
50      40
51      12
52       0
53       0
54       2
55      62
56       8
57       5
58    3239
Name: 11/16/20, dtype: int64

In [204]:
df_Hospitalization_new['NewHosp14AVE'] = (df_Hospitalization_new.iloc[:,-1] - df_Hospitalization_new.iloc[:,-15])/14
df_Hospitalization_new['Hosp14AVE'] = df_Hospitalization_new.iloc[:,-15:-1].mean(axis=1)
df_Hospitalization_new['Hosp7AVE'] = df_Hospitalization_new.iloc[:,-9:-2].mean(axis=1)
df_Hospitalization_new.head()

,ID,County,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,2020-09-20 00:00:00,2020-09-21 00:00:00,...,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,NewHosp14AVE,Hosp14AVE,Hosp7AVE
0,0,ALACHUA,87,83,89,77,68,70,69,72,...,73,74,74,69,70,72,83,1.000000,68.357143,73.571429
1,1,BAKER,21,12,12,15,16,16,16,13,...,1,0,1,1,1,0,0,-0.214286,1.000000,0.571429
2,2,BAY,24,25,26,29,27,27,28,24,...,23,22,31,28,29,23,24,0.428571,21.642857,25.714286
3,3,BRADFORD,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
4,4,BREVARD,58,53,46,42,44,50,46,43,...,81,72,74,78,72,73,79,0.857143,72.285714,75.571429


In [205]:
df_case_merge = pd.merge(df_NewPositive_new,df_NewTested_new, left_on = ['County_1'], right_on = ['County_1'], how="outer")
df_case_merge

,DEPCODE_x,COUNTY_x,COUNTYNAME_x,County_1,2020-09-14 00:00:00_x,2020-09-15 00:00:00_x,2020-09-16 00:00:00_x,2020-09-17 00:00:00_x,2020-09-18 00:00:00_x,2020-09-19 00:00:00_x,...,11/09/20_y,11/10/20_y,11/11/20_y,11/12/20_y,11/13/20_y,11/14/20_y,11/15/20_y,11/16/20_y,7dayTested,14dayTested
0,1,1,ALACHUA,Alachua,114,86,146,160,176,172,...,3646,1121,1901,1787,1158,432,432,1941,8772,19083
1,2,3,BAKER,Baker,10,9,11,20,15,13,...,98,178,119,82,60,32,32,128,631,1253
2,3,5,BAY,Bay,13,27,12,21,14,37,...,211,771,623,587,463,455,455,1073,4427,7254
3,4,7,BRADFORD,Bradford,4,6,18,23,15,17,...,49,69,127,40,71,112,112,105,636,1054
4,5,9,BREVARD,Brevard,36,57,47,95,72,90,...,1107,1342,1288,2115,1457,1079,1079,2355,10715,20273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,129,WAKULLA,Wakulla,19,14,8,12,8,14,...,28,154,167,148,158,66,66,317,1076,1892
65,66,131,WALTON,Walton,17,9,14,5,3,8,...,104,195,175,225,163,74,74,394,1300,3984
66,67,133,WASHINGTON,Washington,0,2,3,0,1,3,...,13,42,23,32,52,24,24,91,288,729
67,998,998,A State,State,1827,2701,2340,3309,3239,3603,...,48365,52115,73420,75557,85600,45072,45072,131506,508342,1016668


In [206]:
df_case = df_case_merge[['DEPCODE_x','County_1','COUNTYNAME_x','7dayPos','14dayPos','7dayTested','14dayTested']].copy()
df_case

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested
0,1,Alachua,ALACHUA,455,914,8772,19083
1,2,Baker,BAKER,51,96,631,1253
2,3,Bay,BAY,460,756,4427,7254
3,4,Bradford,BRADFORD,33,60,636,1054
4,5,Brevard,BREVARD,965,1856,10715,20273
...,...,...,...,...,...,...,...
64,65,Wakulla,WAKULLA,88,134,1076,1892
65,66,Walton,WALTON,188,666,1300,3984
66,67,Washington,WASHINGTON,84,136,288,729
67,998,State,A State,41188,76457,508342,1016668


In [207]:
df_case['7daysPRate']=df_case['7dayPos']/df_case['7dayTested']
df_case['14daysPRate']=df_case['14dayPos']/df_case['14dayTested']
df_case['WeekPR_Ratio']=df_case['7daysPRate']/((df_case['14dayPos']-df_case['7dayPos'])/(df_case['14dayTested']-df_case['7dayTested']))
df_case['WeekPosi_Ratio']=df_case['7dayPos']/(df_case['14dayPos']-df_case['7dayPos'])

In [208]:
df_case 

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,WeekPosi_Ratio
0,1,Alachua,ALACHUA,455,914,8772,19083,0.051870,0.047896,1.165201,0.991285
1,2,Baker,BAKER,51,96,631,1253,0.080824,0.076616,1.117169,1.133333
2,3,Bay,BAY,460,756,4427,7254,0.103908,0.104218,0.992390,1.554054
3,4,Bradford,BRADFORD,33,60,636,1054,0.051887,0.056926,0.803284,1.222222
4,5,Brevard,BREVARD,965,1856,10715,20273,0.090061,0.091550,0.966105,1.083053
...,...,...,...,...,...,...,...,...,...,...,...
64,65,Wakulla,WAKULLA,88,134,1076,1892,0.081784,0.070825,1.450784,1.913043
65,66,Walton,WALTON,188,666,1300,3984,0.144615,0.167169,0.812024,0.393305
66,67,Washington,WASHINGTON,84,136,288,729,0.291667,0.186557,2.473558,1.615385
67,998,State,A State,41188,76457,508342,1016668,0.081024,0.075204,1.167788,1.167824


In [209]:
df_hosp_14 = df_Hospitalization_new[['County','NewHosp14AVE','Hosp14AVE','Hosp7AVE']].copy()
df_hosp_14new = df_hosp_14.astype({'NewHosp14AVE': 'int64','Hosp14AVE': 'int64'})
df_hosp_14new.head()

,County,NewHosp14AVE,Hosp14AVE,Hosp7AVE
0,ALACHUA,1,68,73.571429
1,BAKER,0,1,0.571429
2,BAY,0,21,25.714286
3,BRADFORD,0,0,0.000000
4,BREVARD,0,72,75.571429


In [210]:
a = set(df_case['COUNTYNAME_x'].values)
b = set(df_hosp_14new['County'].values)

In [211]:
b.difference(a)

{'MIAMI-DADE'}

In [212]:
df_hosp_14new.loc[df_hosp_14new['County']=='MIAMI-DADE','County']='DADE'

In [213]:
a = set(df_case['COUNTYNAME_x'].values)
b = set(df_hosp_14new['County'].values)
b.difference(a)

set()

In [214]:
df_merge = pd.merge(df_case, df_hosp_14new, left_on=['COUNTYNAME_x'],right_on= ['County'], how = 'left')
df_merge['WeekHosp_Ratio']=df_merge['Hosp7AVE']/(df_merge['Hosp14AVE']*14-df_merge['Hosp7AVE']*7)*7
df_merge.head(len(df_merge))

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,WeekPosi_Ratio,County,NewHosp14AVE,Hosp14AVE,Hosp7AVE,WeekHosp_Ratio
0,1,Alachua,ALACHUA,455,914,8772,19083,0.051870,0.047896,1.165201,0.991285,ALACHUA,1.0,68.0,73.571429,1.178490
1,2,Baker,BAKER,51,96,631,1253,0.080824,0.076616,1.117169,1.133333,BAKER,0.0,1.0,0.571429,0.400000
2,3,Bay,BAY,460,756,4427,7254,0.103908,0.104218,0.992390,1.554054,BAY,0.0,21.0,25.714286,1.578947
3,4,Bradford,BRADFORD,33,60,636,1054,0.051887,0.056926,0.803284,1.222222,BRADFORD,0.0,0.0,0.000000,NaN
4,5,Brevard,BREVARD,965,1856,10715,20273,0.090061,0.091550,0.966105,1.083053,BREVARD,0.0,72.0,75.571429,1.104384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,Wakulla,WAKULLA,88,134,1076,1892,0.081784,0.070825,1.450784,1.913043,NaN,NaN,NaN,NaN,NaN
65,66,Walton,WALTON,188,666,1300,3984,0.144615,0.167169,0.812024,0.393305,WALTON,0.0,10.0,11.571429,1.372881
66,67,Washington,WASHINGTON,84,136,288,729,0.291667,0.186557,2.473558,1.615385,WASHINGTON,0.0,3.0,4.571429,3.200000
67,998,State,A State,41188,76457,508342,1016668,0.081024,0.075204,1.167788,1.167824,A State,54.0,2871.0,3112.714286,1.183863


In [215]:
df_sort = df_merge.sort_values(by=['DEPCODE_x'])
df_sort['County_1']=df_sort['County_1'].str.title()
df_sort

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,WeekPosi_Ratio,County,NewHosp14AVE,Hosp14AVE,Hosp7AVE,WeekHosp_Ratio
0,1,Alachua,ALACHUA,455,914,8772,19083,0.051870,0.047896,1.165201,0.991285,ALACHUA,1.0,68.0,73.571429,1.178490
1,2,Baker,BAKER,51,96,631,1253,0.080824,0.076616,1.117169,1.133333,BAKER,0.0,1.0,0.571429,0.400000
2,3,Bay,BAY,460,756,4427,7254,0.103908,0.104218,0.992390,1.554054,BAY,0.0,21.0,25.714286,1.578947
3,4,Bradford,BRADFORD,33,60,636,1054,0.051887,0.056926,0.803284,1.222222,BRADFORD,0.0,0.0,0.000000,NaN
4,5,Brevard,BREVARD,965,1856,10715,20273,0.090061,0.091550,0.966105,1.083053,BREVARD,0.0,72.0,75.571429,1.104384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,Wakulla,WAKULLA,88,134,1076,1892,0.081784,0.070825,1.450784,1.913043,NaN,NaN,NaN,NaN,NaN
65,66,Walton,WALTON,188,666,1300,3984,0.144615,0.167169,0.812024,0.393305,WALTON,0.0,10.0,11.571429,1.372881
66,67,Washington,WASHINGTON,84,136,288,729,0.291667,0.186557,2.473558,1.615385,WASHINGTON,0.0,3.0,4.571429,3.200000
67,998,State,A State,41188,76457,508342,1016668,0.081024,0.075204,1.167788,1.167824,A State,54.0,2871.0,3112.714286,1.183863


In [216]:
df_sort_extract = df_sort.drop(columns=['County'])
df_sort_drop_unknown = df_sort_extract.drop([68])
df_sort_drop_unknown

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,WeekPosi_Ratio,NewHosp14AVE,Hosp14AVE,Hosp7AVE,WeekHosp_Ratio
0,1,Alachua,ALACHUA,455,914,8772,19083,0.051870,0.047896,1.165201,0.991285,1.0,68.0,73.571429,1.178490
1,2,Baker,BAKER,51,96,631,1253,0.080824,0.076616,1.117169,1.133333,0.0,1.0,0.571429,0.400000
2,3,Bay,BAY,460,756,4427,7254,0.103908,0.104218,0.992390,1.554054,0.0,21.0,25.714286,1.578947
3,4,Bradford,BRADFORD,33,60,636,1054,0.051887,0.056926,0.803284,1.222222,0.0,0.0,0.000000,NaN
4,5,Brevard,BREVARD,965,1856,10715,20273,0.090061,0.091550,0.966105,1.083053,0.0,72.0,75.571429,1.104384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,VOLUSIA,536,1220,7503,15864,0.071438,0.076904,0.873237,0.783626,0.0,59.0,60.285714,1.044554
64,65,Wakulla,WAKULLA,88,134,1076,1892,0.081784,0.070825,1.450784,1.913043,NaN,NaN,NaN,NaN
65,66,Walton,WALTON,188,666,1300,3984,0.144615,0.167169,0.812024,0.393305,0.0,10.0,11.571429,1.372881
66,67,Washington,WASHINGTON,84,136,288,729,0.291667,0.186557,2.473558,1.615385,0.0,3.0,4.571429,3.200000


In [217]:
print("Saving COVID_14_update{}.xlsx in results/".format(yearDate))
df_sort_drop_unknown.to_excel(os.path.join(result_folder,"COVID_14_update{}.xlsx".format(yearDate)))

Saving COVID_14_update111620.xlsx in results/


In [218]:
gdf_county_polygon = gpd.read_file("shp/Florida_Counties.shp")
gdf_county_polygon.head()

,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,21,041,Gilchrist,2000-05-16,9.908353e+09,4.873000e+05,"POLYGON ((-82.65814 29.83028, -82.65803 29.822..."
1,2,54,107,Putnam,2000-05-16,2.305869e+10,7.629677e+05,"POLYGON ((-81.58084 29.83956, -81.57893 29.835..."
2,3,62,123,Taylor,2000-05-16,2.891747e+10,8.772527e+05,"MULTIPOLYGON (((-83.73037 30.30358, -83.70368 ..."
3,4,46,091,Okaloosa,2000-05-16,2.562159e+10,1.087058e+06,"MULTIPOLYGON (((-86.39159 30.64970, -86.39175 ..."
4,5,7,013,Calhoun,2000-05-16,1.604809e+10,6.313440e+05,"POLYGON ((-84.93266 30.60637, -84.93287 30.605..."


In [219]:
a = set(df_sort_drop_unknown['County_1'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

{'State'}

In [220]:
b.difference(a)

set()

In [221]:
df_merge_poly = pd.merge(df_sort_drop_unknown, gdf_county_polygon, left_on = ['County_1'],right_on= ['COUNTYNAME'], how = 'outer')
df_merge_poly.head(len(df_merge_poly))

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,...,Hosp7AVE,WeekHosp_Ratio,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,Alachua,ALACHUA,455,914,8772,19083,0.051870,0.047896,1.165201,...,73.571429,1.178490,66.0,1.0,001,Alachua,2000-05-16,2.702324e+10,8.927266e+05,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,2,Baker,BAKER,51,96,631,1253,0.080824,0.076616,1.117169,...,0.571429,0.400000,17.0,2.0,003,Baker,2000-05-16,1.636216e+10,6.027173e+05,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,3,Bay,BAY,460,756,4427,7254,0.103908,0.104218,0.992390,...,25.714286,1.578947,15.0,3.0,005,Bay,2000-05-16,2.084232e+10,1.834701e+06,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,4,Bradford,BRADFORD,33,60,636,1054,0.051887,0.056926,0.803284,...,0.000000,NaN,40.0,4.0,007,Bradford,2000-05-16,8.359863e+09,5.483267e+05,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,5,Brevard,BREVARD,965,1856,10715,20273,0.090061,0.091550,0.966105,...,75.571429,1.104384,42.0,5.0,009,Brevard,2000-05-16,2.893695e+10,2.600567e+06,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,VOLUSIA,536,1220,7503,15864,0.071438,0.076904,0.873237,...,60.285714,1.044554,25.0,64.0,127,Volusia,2000-05-16,3.364565e+10,1.837936e+06,"MULTIPOLYGON (((-81.09705 29.35180, -81.09292 ..."
64,65,Wakulla,WAKULLA,88,134,1076,1892,0.081784,0.070825,1.450784,...,NaN,NaN,41.0,65.0,129,Wakulla,2000-05-16,1.702251e+10,9.167339e+05,"MULTIPOLYGON (((-84.41197 30.30244, -84.37858 ..."
65,66,Walton,WALTON,188,666,1300,3984,0.144615,0.167169,0.812024,...,11.571429,1.372881,51.0,66.0,131,Walton,2000-05-16,2.926099e+10,1.216747e+06,"POLYGON ((-86.03523 30.98825, -86.03532 30.982..."
66,67,Washington,WASHINGTON,84,136,288,729,0.291667,0.186557,2.473558,...,4.571429,3.200000,53.0,67.0,133,Washington,2000-05-16,1.721229e+10,8.010119e+05,"POLYGON ((-85.43543 30.56727, -85.46843 30.567..."


In [222]:
df_merge_poly_drop = df_merge_poly.drop(columns=['DEPCODE','COUNTYNAME','COUNTY','OBJECTID','ShapeSTAre','ShapeSTLen','DATESTAMP'])
df_merge_poly_clean = df_merge_poly_drop.rename(columns={'DEPCODE_x':'DEPCODE','County_1':'County','COUNTYNAME_x':'COUNTYNAME'})
df_merge_poly_clean

,DEPCODE,County,COUNTYNAME,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,WeekPosi_Ratio,NewHosp14AVE,Hosp14AVE,Hosp7AVE,WeekHosp_Ratio,geometry
0,1,Alachua,ALACHUA,455,914,8772,19083,0.051870,0.047896,1.165201,0.991285,1.0,68.0,73.571429,1.178490,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,2,Baker,BAKER,51,96,631,1253,0.080824,0.076616,1.117169,1.133333,0.0,1.0,0.571429,0.400000,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,3,Bay,BAY,460,756,4427,7254,0.103908,0.104218,0.992390,1.554054,0.0,21.0,25.714286,1.578947,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,4,Bradford,BRADFORD,33,60,636,1054,0.051887,0.056926,0.803284,1.222222,0.0,0.0,0.000000,NaN,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,5,Brevard,BREVARD,965,1856,10715,20273,0.090061,0.091550,0.966105,1.083053,0.0,72.0,75.571429,1.104384,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,VOLUSIA,536,1220,7503,15864,0.071438,0.076904,0.873237,0.783626,0.0,59.0,60.285714,1.044554,"MULTIPOLYGON (((-81.09705 29.35180, -81.09292 ..."
64,65,Wakulla,WAKULLA,88,134,1076,1892,0.081784,0.070825,1.450784,1.913043,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-84.41197 30.30244, -84.37858 ..."
65,66,Walton,WALTON,188,666,1300,3984,0.144615,0.167169,0.812024,0.393305,0.0,10.0,11.571429,1.372881,"POLYGON ((-86.03523 30.98825, -86.03532 30.982..."
66,67,Washington,WASHINGTON,84,136,288,729,0.291667,0.186557,2.473558,1.615385,0.0,3.0,4.571429,3.200000,"POLYGON ((-85.43543 30.56727, -85.46843 30.567..."


In [223]:
shpfile_name = "covid_14_metrics{}".format(yearDate)
print("Saving {}.shp in results/".format(shpfile_name))
gdf_merge_poly = gpd.GeoDataFrame(df_merge_poly_clean, geometry='geometry',crs="EPSG:4326")
gdf_merge_poly.to_file(driver = 'ESRI Shapefile', filename= os.path.join(result_folder,"{}.shp".format(shpfile_name)))

Saving covid_14_metrics111620.shp in results/


In [224]:
print("Saving {}.zip in publish/".format(shpfile_name))
with ZipFile(os.path.join('publish', '{}.zip'.format(shpfile_name)), 'w') as zipObj:
    for file in os.listdir(result_folder):
        if shpfile_name in file:
            file_path = os.path.join(result_folder, file)
            print(file)
            try:
                zipObj.write(file_path, os.path.basename(file_path))
            except OSError as err:
                print("OS error: {0}".format(err))   

Saving covid_14_metrics111620.zip in publish/
covid_14_metrics111620.shp
covid_14_metrics111620.cpg
covid_14_metrics111620.shx
covid_14_metrics111620.dbf
covid_14_metrics111620.prj


In [225]:
print(str(datetime.now()) + " Done with 6_COVID_14day_Metrics_Process")

2020-11-16 12:54:15.273905 Done with 6_COVID_14day_Metrics_Process
